In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../src")))

In [2]:
import pandas as pd

In [3]:
from courses_scheduler.planning import AcademicPlan
from courses_scheduler.combinations import OptionsSet
from courses_scheduler.objects import (
    AcademicDiscipline,
    Classroom,
    Students,
    Teacher,
    TimeSlot,
)

# Подготовка

In [4]:
def make_df(options: OptionsSet, choosen_idx: list[int]) -> pd.DataFrame:
    columns = ["time_slot", "students", "teacher", "classroom", "discipline"]
    res = []
    for i, (ts, s, t, c, d) in enumerate(options):
        if i in choosen_idx:
            res.append((
                ts.__repr__(),
                s.__repr__(),
                t.__repr__(),
                c.__repr__(),
                d.__repr__(),
            ))
    return pd.DataFrame(res, columns=columns)

In [5]:
s = {
    k: Students(group_id=k)
    for k in "ABCDEFG"
}

In [6]:
t = {
    k: Teacher(name=k)
    for k in "ABCDEFG"
}

In [7]:
c = {
    k: Classroom(room_number=str(k))
    for k in range(20)
}

In [8]:
d = {
    k: AcademicDiscipline(title=k)
    for k in "ABCDEFG"
}

In [9]:
ts = {
    k: TimeSlot(date_from=k)
    for k in range(20)
}

# Простейший сценарий

In [10]:
ap = AcademicPlan(
    students_workload={s["A"]: {d["A"]: 1}},
    teachers_workload={t["A"]: {d["A"]: 1}},
    available_classrooms={c[1], c[2]},
    available_time_slots=ts.values(),
)

ap.optimizer.model.solve(disp=False)

2024-09-21 21:03:18.618 | INFO     | courses_scheduler.planning:build_optimizer:71 - Start building optimizer
2024-09-21 21:03:18.619 | INFO     | courses_scheduler.planning:build_options:51 - Start building options
100%|██████████| 1/1 [00:00<00:00, 15141.89it/s]
2024-09-21 21:03:18.624 | INFO     | courses_scheduler.optimization:__init__:34 - Start declaring optimized vars
40it [00:00, 47976.03it/s]
2024-09-21 21:03:18.627 | INFO     | courses_scheduler.optimization:__init__:46 - Start declaring students_discipline equations
100%|██████████| 1/1 [00:00<00:00, 5983.32it/s]
2024-09-21 21:03:18.629 | INFO     | courses_scheduler.optimization:__init__:62 - Start declaring teacher_discipline equations
100%|██████████| 1/1 [00:00<00:00, 3048.19it/s]


In [11]:
df = make_df(ap.optimizer.options, ap.optimizer.choosen_options_idx)
df.head()

,time_slot,students,teacher,classroom,discipline
0,0,A,A,2,A


# Несколько учебных групп

In [12]:
ap = AcademicPlan(
    students_workload={s["A"]: {d["A"]: 1}, s["B"]: {d["A"]: 1}},
    teachers_workload={t["A"]: {d["A"]: 2}},
    available_classrooms={c[1], c[2]},
    available_time_slots=ts.values(),
)

ap.optimizer.model.solve(disp=False)

2024-09-21 21:03:18.707 | INFO     | courses_scheduler.planning:build_optimizer:71 - Start building optimizer
2024-09-21 21:03:18.708 | INFO     | courses_scheduler.planning:build_options:51 - Start building options
100%|██████████| 1/1 [00:00<00:00, 13486.51it/s]
2024-09-21 21:03:18.710 | INFO     | courses_scheduler.optimization:__init__:34 - Start declaring optimized vars
80it [00:00, 113168.40it/s]
2024-09-21 21:03:18.712 | INFO     | courses_scheduler.optimization:__init__:46 - Start declaring students_discipline equations
100%|██████████| 2/2 [00:00<00:00, 8346.87it/s]
2024-09-21 21:03:18.713 | INFO     | courses_scheduler.optimization:__init__:62 - Start declaring teacher_discipline equations
100%|██████████| 1/1 [00:00<00:00, 5825.42it/s]


In [13]:
df = make_df(ap.optimizer.options, ap.optimizer.choosen_options_idx)
df.head()

,time_slot,students,teacher,classroom,discipline
0,0,A,A,2,A
1,0,B,A,2,A


# Несколько преподавателей

In [14]:
ap = AcademicPlan(
    students_workload={s["A"]: {d["A"]: 1}},
    teachers_workload={t["A"]: {d["A"]: 1}, t["B"]: {d["A"]: 1}},
    available_classrooms={c[1], c[2]},
    available_time_slots=ts.values(),
)

ap.optimizer.model.solve(disp=False)

2024-09-21 21:03:18.815 | INFO     | courses_scheduler.planning:build_optimizer:71 - Start building optimizer
2024-09-21 21:03:18.816 | INFO     | courses_scheduler.planning:build_options:51 - Start building options
100%|██████████| 1/1 [00:00<00:00, 16448.25it/s]
2024-09-21 21:03:18.818 | INFO     | courses_scheduler.optimization:__init__:34 - Start declaring optimized vars
80it [00:00, 125766.24it/s]
2024-09-21 21:03:18.820 | INFO     | courses_scheduler.optimization:__init__:46 - Start declaring students_discipline equations
100%|██████████| 1/1 [00:00<00:00, 5433.04it/s]
2024-09-21 21:03:18.821 | INFO     | courses_scheduler.optimization:__init__:62 - Start declaring teacher_discipline equations
100%|██████████| 2/2 [00:00<00:00, 9500.12it/s]


In [15]:
df = make_df(ap.optimizer.options, ap.optimizer.choosen_options_idx)
df.head()

,time_slot,students,teacher,classroom,discipline
0,0,A,A,2,A


# Несколько дисциплин

In [16]:
ap = AcademicPlan(
    students_workload={s["A"]: {d["A"]: 1, d["B"]: 1}},
    teachers_workload={t["A"]: {d["A"]: 1, d["B"]: 1}},
    available_classrooms={c[1], c[2]},
    available_time_slots=ts.values(),
)

ap.optimizer.model.solve(disp=False)

2024-09-21 21:03:18.901 | INFO     | courses_scheduler.planning:build_optimizer:71 - Start building optimizer
2024-09-21 21:03:18.902 | INFO     | courses_scheduler.planning:build_options:51 - Start building options
100%|██████████| 2/2 [00:00<00:00, 54471.48it/s]
2024-09-21 21:03:18.904 | INFO     | courses_scheduler.optimization:__init__:34 - Start declaring optimized vars
80it [00:00, 128315.23it/s]
2024-09-21 21:03:18.906 | INFO     | courses_scheduler.optimization:__init__:46 - Start declaring students_discipline equations
100%|██████████| 2/2 [00:00<00:00, 10082.46it/s]
2024-09-21 21:03:18.907 | INFO     | courses_scheduler.optimization:__init__:62 - Start declaring teacher_discipline equations
100%|██████████| 2/2 [00:00<00:00, 11140.25it/s]


In [17]:
df = make_df(ap.optimizer.options, ap.optimizer.choosen_options_idx)
df.head()

,time_slot,students,teacher,classroom,discipline
0,0,A,A,2,B
1,0,A,A,2,A


# Несколько предметов у разных групп

In [18]:
ap = AcademicPlan(
    students_workload={s["A"]: {d["A"]: 1}, s["B"]: {d["B"]: 1}},
    teachers_workload={t["A"]: {d["A"]: 1, d["B"]: 1}},
    available_classrooms={c[1], c[2]},
    available_time_slots=ts.values(),
)

ap.optimizer.model.solve(disp=False)

2024-09-21 21:03:18.999 | INFO     | courses_scheduler.planning:build_optimizer:71 - Start building optimizer
2024-09-21 21:03:19.001 | INFO     | courses_scheduler.planning:build_options:51 - Start building options
100%|██████████| 2/2 [00:00<00:00, 25811.10it/s]
2024-09-21 21:03:19.004 | INFO     | courses_scheduler.optimization:__init__:34 - Start declaring optimized vars
80it [00:00, 48113.61it/s]
2024-09-21 21:03:19.009 | INFO     | courses_scheduler.optimization:__init__:46 - Start declaring students_discipline equations
100%|██████████| 2/2 [00:00<00:00, 6579.30it/s]
2024-09-21 21:03:19.011 | INFO     | courses_scheduler.optimization:__init__:62 - Start declaring teacher_discipline equations
100%|██████████| 2/2 [00:00<00:00, 9039.45it/s]


In [19]:
df = make_df(ap.optimizer.options, ap.optimizer.choosen_options_idx)
df.head()

,time_slot,students,teacher,classroom,discipline
0,0,B,A,2,B
1,0,A,A,2,A


# Нехватка ресурсов

In [20]:
ap = AcademicPlan(
    students_workload={s["A"]: {d["A"]: 2}},
    teachers_workload={t["A"]: {d["A"]: 1}},
    available_classrooms={c[1], c[2]},
    available_time_slots=ts.values(),
)

ap.optimizer.model.solve(disp=False)

2024-09-21 21:03:19.100 | INFO     | courses_scheduler.planning:build_optimizer:71 - Start building optimizer
2024-09-21 21:03:19.101 | INFO     | courses_scheduler.planning:build_options:51 - Start building options
100%|██████████| 1/1 [00:00<00:00, 18808.54it/s]
2024-09-21 21:03:19.103 | INFO     | courses_scheduler.optimization:__init__:34 - Start declaring optimized vars
40it [00:00, 73262.95it/s]
2024-09-21 21:03:19.104 | INFO     | courses_scheduler.optimization:__init__:46 - Start declaring students_discipline equations
100%|██████████| 1/1 [00:00<00:00, 3938.31it/s]
2024-09-21 21:03:19.107 | INFO     | courses_scheduler.optimization:__init__:62 - Start declaring teacher_discipline equations
100%|██████████| 1/1 [00:00<00:00, 2690.38it/s]


Exception: @error: Solution Not Found


# Большие объемы

In [30]:
ts_large = {
    k: TimeSlot(date_from=k)
    for k in range(50)
}

In [33]:
%%time
ap = AcademicPlan(
    students_workload={s["A"]: {d_i: 4 for d_i in d.values()}},
    teachers_workload={t_i: {d_i: 2 for d_i in d.values()} for t_i in t.values()},
    available_classrooms={c[1]},
    available_time_slots=ts_large.values(),
)

2024-09-21 21:07:16.520 | INFO     | courses_scheduler.planning:build_optimizer:71 - Start building optimizer
2024-09-21 21:07:16.522 | INFO     | courses_scheduler.planning:build_options:51 - Start building options
100%|██████████| 7/7 [00:00<00:00, 6270.85it/s]
2024-09-21 21:07:16.534 | INFO     | courses_scheduler.optimization:__init__:34 - Start declaring optimized vars
2450it [00:00, 75608.63it/s]
2024-09-21 21:07:16.575 | INFO     | courses_scheduler.optimization:__init__:46 - Start declaring students_discipline equations
100%|██████████| 7/7 [00:00<00:00, 229.80it/s]
2024-09-21 21:07:16.612 | INFO     | courses_scheduler.optimization:__init__:62 - Start declaring teacher_discipline equations
100%|██████████| 49/49 [00:00<00:00, 1683.74it/s]


CPU times: user 206 ms, sys: 5.02 ms, total: 211 ms
Wall time: 208 ms


In [34]:
%%time
ap.optimizer.model.solve(disp=True, debug=False)

 ----------------------------------------------------------------
 APMonitor, Version 1.0.3
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :           58
   Constants    :            0
   Variables    :         5007
   Intermediates:            0
   Connections  :         9858
   Equations    :         2507
   Residuals    :         2507
 
 Reduction analysis complete
 Eliminated           49  of         5007  variables
 Eliminated           49  of         2564  equations
 Number of state variables:           4958
 Number of total equations: -         2515
 Number of slack variables: -           49
 ---------------------------------------
 Degrees of freedom       :           2394
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.05 NL

In [35]:
df = make_df(ap.optimizer.options, ap.optimizer.choosen_options_idx)
df.head()

,time_slot,students,teacher,classroom,discipline
0,0,A,A,1,F
1,0,A,B,1,F
2,0,A,C,1,F
3,0,A,D,1,F
4,0,A,A,1,G


In [37]:
df.value_counts(["teacher", "discipline"])

teacher  discipline
A        A             1
         B             1
         C             1
         D             1
         E             1
         F             1
         G             1
B        A             1
         B             1
         C             1
         D             1
         E             1
         F             1
         G             1
C        A             1
         B             1
         C             1
         D             1
         E             1
         F             1
         G             1
D        A             1
         B             1
         C             1
         D             1
         E             1
         F             1
         G             1
Name: count, dtype: int64

In [24]:
import os
import shutil

tmp_dir = '/tmp'
for folder in os.listdir(tmp_dir):
    if 'gk_model' in folder:
        print(folder)
        shutil.rmtree(os.path.join(tmp_dir, folder))

tmp3prjsjopgk_model6
tmpivb4li68gk_model2
tmpx3f_0wuvgk_model3
tmpgngh40a4gk_model0
tmp25k6_irlgk_model1
tmpl75fnymggk_model4
tmppyz8dprsgk_model5
